In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

체크 사항  

휴일인지 아닌지  

In [2]:
df = pd.read_csv('./data/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
df.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [3]:
df['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

### 전처리

####  1.1 '구분' 칼럼 전처리

In [3]:
d_map = {}

for i, d in enumerate(df['구분'].unique()):
    d_map[d] = i 

df['구분'] = df['구분'].map(d_map)
df.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,0,2497.129
1,2013-01-01,2,0,2363.265
2,2013-01-01,3,0,2258.505
3,2013-01-01,4,0,2243.969
4,2013-01-01,5,0,2344.105


In [5]:
df['연월일'] = pd.to_datetime(df['연월일'])
df['year'] = df['연월일'].dt.year
df['month'] = df['연월일'].dt.month
df['day'] = df['연월일'].dt.day
df['weekday'] = df['연월일'].dt.weekday    # 요일(0부터 월요일)
# df['weekday_name'] = df['연월일'].dt.day_name

df.head()

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1


In [25]:
df.groupby(['구분']).mean()

,시간,공급량,year,month,day,weekday
구분,,,,,,
0,12.5,1167.014483,2015.500228,6.523962,15.726609,3.0
1,12.5,958.564621,2015.500228,6.523962,15.726609,3.0
2,12.5,131.585481,2015.500228,6.523962,15.726609,3.0
3,12.5,664.626452,2015.500228,6.523962,15.726609,3.0
4,12.5,1395.356673,2015.500228,6.523962,15.726609,3.0
5,12.5,2070.685900,2015.500228,6.523962,15.726609,3.0
6,12.5,248.866646,2015.500228,6.523962,15.726609,3.0


In [32]:
df.groupby('month')['공급량'].mean()

month
1     1833.304906
2     1664.005874
3     1209.032748
4      805.711471
5      553.855529
6      482.855214
7      475.608166
8      448.937382
9      466.313754
10     637.341657
11    1136.587102
12    1700.016547
Name: 공급량, dtype: float64

In [39]:
df.groupby('시간')['공급량'].mean()

시간
1      821.285596
2      683.393277
3      612.326706
4      630.644148
5      715.591729
6      788.153478
7      955.432293
8     1169.864435
9     1188.272911
10    1143.701014
11    1117.409266
12    1029.675001
13     976.638568
14     942.007362
15     885.661072
16     857.178031
17     879.557707
18     970.896986
19    1099.290023
20    1139.231434
21    1117.785868
22    1079.574160
23    1011.378763
24     939.451053
Name: 공급량, dtype: float64

In [38]:
df.groupby('weekday')['공급량'].mean()

weekday
0    960.247853
1    983.333307
2    982.156862
3    987.518102
4    979.038016
5    895.148338
6    849.257779
Name: 공급량, dtype: float64

In [40]:
df['meanOfTime'] = df.groupby('시간')['공급량'].transform('mean')

In [29]:
df['meanOfMonth'] = df.groupby('month')['공급량'].transform('mean')

In [35]:
df['meanOfArea'] = df.groupby('구분')['공급량'].transform('mean')

In [41]:
df.head()

,연월일,시간,구분,공급량,year,month,day,weekday,meanOfMonth,meanOfArea,meanOfTime
0,2013-01-01,1,0,2497.129,2013,1,1,1,1833.304906,1167.014483,821.285596
1,2013-01-01,2,0,2363.265,2013,1,1,1,1833.304906,1167.014483,683.393277
2,2013-01-01,3,0,2258.505,2013,1,1,1,1833.304906,1167.014483,612.326706
3,2013-01-01,4,0,2243.969,2013,1,1,1,1833.304906,1167.014483,630.644148
4,2013-01-01,5,0,2344.105,2013,1,1,1,1833.304906,1167.014483,715.591729


In [30]:
df['meanOfMonth']

0         1833.304906
1         1833.304906
2         1833.304906
3         1833.304906
4         1833.304906
             ...     
368083    1700.016547
368084    1700.016547
368085    1700.016547
368086    1700.016547
368087    1700.016547
Name: meanOfMonth, Length: 368088, dtype: float64

In [43]:
train_years = [2013, 2014, 2015, 2016, 2017]
val_years = [2018]

In [44]:
train = df[df['year'].isin(train_years)]
val = df[df['year'].isin(val_years)]

In [45]:
features = ['구분', 'month', 'day', 'weekday', '시간', 'meanOfTime', 'meanOfMonth', 'meanOfArea']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [46]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = { 
    'objective': 'regression',
    'metric': 'mae',
    'seed': 42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001891 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 8
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 218.354
[40]	valid_0's l1: 162.325
[60]	valid_0's l1: 150.318
[80]	valid_0's l1: 146.929
[100]	valid_0's l1: 146.328
[120]	valid_0's l1: 145.247
[140]	valid_0's l1: 144.256
[160]	valid_0's l1: 143.665
Early stopping, best iteration is:
[169]	valid_0's l1: 143.522


&nbsp;  
&nbsp;  
===================== test =======================  

In [47]:
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [48]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [57]:
test.tail()

,일자|시간|구분,일자,시간,구분,year,month,day,weekday
15115,2019-03-31 20 H,2019-03-31,20,6,2019,3,31,6
15116,2019-03-31 21 H,2019-03-31,21,6,2019,3,31,6
15117,2019-03-31 22 H,2019-03-31,22,6,2019,3,31,6
15118,2019-03-31 23 H,2019-03-31,23,6,2019,3,31,6
15119,2019-03-31 24 H,2019-03-31,24,6,2019,3,31,6


In [49]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [50]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [51]:
test['구분'] = test['구분'].map(d_map)

In [54]:
test_features = ['구분', 'month', 'day', 'weekday', '시간']
test_x = test[test_features]

In [56]:
test_x.head()

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5


In [55]:
preds = model.predict(test_x)

LightGBMError: The number of features in data (5) is not the same as it was in training data (8).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [16]:
submission['공급량'] = preds

In [17]:
submission.to_csv('baseline.csv', index=False)